In [1]:
import pandas as pd
from database import Database
from config import Config

ModuleNotFoundError: No module named 'database'

In [3]:
from config import Config

ModuleNotFoundError: No module named 'config'

In [ ]:
bookings = pd.read_csv("bookings.csv")

db = Database()

hotel_mapping = db.get_hotel_mapping()
room_mapping = db.get_room_mapping()
meal_mapping = db.get_meal_mapping()
operator_mapping = db.get_operator_mapping()

In [ ]:
hotel_mapping = dict(zip(hotels["external_code"], hotels["hotel_id"]))
room_mapping = dict(
    zip(
        rooms.apply(lambda x: (x["external_code"], x["hotel_id"]), axis=1),
        rooms["id"],
    )
)
meal_mapping = dict(zip(meals["code"], meals["id"]))
operator_mapping = dict(zip(operators["external_code"], operators["operator_id"]))

In [ ]:
bookings["external_code_hotel"] = bookings["hotel_id"]
bookings["hotel_id"] = pd.to_numeric(
    bookings["external_code_hotel"].map(hotel_mapping), errors="coerce"
)
bookings["hotel_id"] = bookings["hotel_id"].astype("Int64")

In [ ]:
bookings["external_code_room"] = bookings["room_code"]
bookings["room_id"] = bookings.apply(
    lambda x: room_mapping.get(
        (
            x["external_code_room"],
            x["hotel_id"],
        )
    ),
    axis=1,
)
bookings["room_id"] = pd.to_numeric(bookings["room_id"], errors="coerce").astype(
    "Int64"
)

In [ ]:
bookings["external_code_meal"] = bookings["meal"]
bookings["meal_id"] = bookings["external_code_meal"].map(meal_mapping)
bookings["meal_id"] = pd.to_numeric(bookings["meal_id"], errors="coerce").astype(
    "Int64"
)

In [ ]:
bookings["external_code_operator"] = bookings["operator_id"]
bookings["operator_id"] = bookings["external_code_operator"].map(operator_mapping)
bookings["operator_id"] = pd.to_numeric(
    bookings["operator_id"], errors="coerce"
).astype("Int64")

In [ ]:
bookings.head()

In [ ]:
bookings.to_csv("temp.csv", index=False)

In [ ]:
import os
from typing import Union, get_type_hints

import psycopg2


class AppConfigError(Exception):
    pass


def _parse_bool(val: Union[str, bool]) -> bool:  # pylint: disable=E1136
    return val if type(val) == bool else val.lower() in ["true", "yes", "1"]


# AppConfig class with required fields, default values, type checking, and typecasting for int and bool values


class AppConfig:
    SSRS_BASE_URL: str
    SSRS_USERNAME: str
    SSRS_PASSWORD: str
    DATABASE_URI: str
    MAIL_SERVER: str
    MAIL_PORT: int
    MAIL_USE_TLS: bool = True
    MAIL_USERNAME: str
    MAIL_PASSWORD: str
    FTP_SERVER: str
    FTP_USERNAME: str
    FTP_PASSWORD: str

    """
    Map environment variables to class fields according to these rules:
      - Field won't be parsed unless it has a type annotation
      - Field will be skipped if not in all caps
      - Class field and environment variable name are the same
    """

    def __init__(self, env):
        for field in self.__annotations__:
            if not field.isupper():
                continue

            # Raise AppConfigError if required field not supplied
            default_value = getattr(self, field, None)
            if default_value is None and env.get(field) is None:
                raise AppConfigError("The {} field is required".format(field))

            # Cast env var value to expected type and raise AppConfigError on failure
            try:
                var_type = get_type_hints(AppConfig)[field]
                if var_type == bool:
                    value = _parse_bool(env.get(field, default_value))
                else:
                    value = var_type(env.get(field, default_value))

                self.__setattr__(field, value)
            except ValueError:
                raise AppConfigError(
                    'Unable to cast value of "{}" to type "{}" for "{}" field'.format(
                        env[field], var_type, field
                    )
                )

    def __repr__(self):
        return str(self.__dict__)


# Expose Config object for app to import
Config = AppConfig(os.environ)